In [ ]:
import import_ipynb
import pandas as pd
import numpy as np
import torch
from datetime import datetime
from datetime import datetime as dt
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler
import pickle

In [ ]:
from feeds import BackFeed,DataFeed
from featfuncs import feat_aug,add_addl_features_feed,add_ta_features_feed,add_sym_feature_feed
from featfuncs import add_global_indices_feed

In [ ]:
from rulestrats import AdaMomCMOADF, GapBet

In [ ]:
def do_nothing(dfD):
    empty={t:0 for t in dfD}
    return empty,empty,empty
def always_buy(dfD):
    buy={t:1 for t in dfD}
    empty={t:0 for t in dfD}
    return buy,empty,empty
def always_sell(dfD):
    sell={t:-1 for t in dfD}
    empty={t:0 for t in dfD}
    return sell,empty,empty

In [ ]:
def get_alt_data():
    global feed
    aD={'gdata':feed.gdata}
    return aD

Host Service via Anvil

In [ ]:
import anvil.server

In [ ]:
# DeepTrader:
anvil.server.connect("STKYCIXUMGFV2CURGMCMOFUV-MBK5ZXFC4MCKYKOE")

In [ ]:
momStrat=MomStrat()

In [ ]:
momStrat=AdaMomCMOADF()

In [ ]:
@anvil.server.callable
def check_entry_batch_mom2(dfD):
    global momStrat
    dfDloc={}
    for t in dfD.keys():
        dfDloc[t]=pd.DataFrame(dfD[t])
    return MomStrat.Check(momStrat,dfDloc)

In [ ]:
@anvil.server.callable
def check_exit_batch_mom2(dfD,posf):
    global momStrat
    posloc=pd.DataFrame(posf)
    dfDloc={}
    for t in dfD.keys():
        dfDloc[t]=pd.DataFrame(dfD[t])
    return MomStrat.Exit(momStrat,dfDloc,posloc)

Strategy Development: ADF Momentum

In [ ]:
from backtest import Backtest
from feeds import BackFeed,DataFeed
from validation import Validate

In [ ]:
data=pd.read_csv('./capvol100.csv')

In [ ]:
tickers=list(data.iloc[0:10]['ticker'].values)

In [ ]:
feed=BackFeed(tickers=tickers,nd=3,nw=1,interval='5m')

In [ ]:
add_addl_features_feed(feed,tickers=feed.tickers)

In [ ]:
add_sym_feature_feed(feed,tickers=feed.tickers)

In [ ]:
add_global_indices_feed(feed)

In [ ]:
import pickle

In [ ]:
# with open('../../temp_data/btfeed.pickle','wb') as f: pickle.dump(feed,f)

In [ ]:
with open('../../temp_data/btfeed.pickle','rb') as f: feed=pickle.load(f)

In [ ]:
gapbet=GapBet(direction=-1)

In [ ]:
gapbet.set_alt_data(alt_data_func=get_alt_data)

In [ ]:
adamom=AdaMomCMOADF()

In [ ]:
import importlib
import sys,os

In [ ]:
importlib.reload(sys.modules['feed_env'])
from feed_env import Episode

In [ ]:
importlib.reload(sys.modules['backtest'])
from backtest import Backtest

In [ ]:
importlib.reload(sys.modules['rulestrats'])
from rulestrats import AdaMomCMOADF,GapBet

In [ ]:
# def save_func(df): return(df.iloc[-1]['CMO_14'])

In [ ]:
bt=Backtest(feed,tickers=feed.tickers,add_features=False,target=.5,stop=.5,txcost=0.001,
            loc_exit=True,scan=True,topk=3,deploy=True,save_dfs=True,
            save_func=None)

In [ ]:
bt.run_all(tickers=feed.tickers,model=gapbet,verbose=False)

In [ ]:
bt.returns

In [ ]:
bt.results